<a href="https://colab.research.google.com/github/Usnish-Mukherjee/Intrusion_Detection/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import numpy as np
from glob import glob
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

In [34]:
nd1 = np.zeros((0,79),dtype=object)
data_set=pd.read_csv('Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv')
nd1=np.vstack((nd1,data_set))

In [35]:
nd1.shape

(331100, 79)

In [36]:
Dt1=nd1[:,:-1].astype(float)

In [37]:
LNMV1=nd1[~np.isnan(Dt1).any(axis=1),-1]
DtNMV1=Dt1[~np.isnan(Dt1).any(axis=1)]
LNMIV1=LNMV1[~np.isinf(DtNMV1).any(axis=1)]
DtNMIV1=DtNMV1[~np.isinf(DtNMV1).any(axis=1)]

In [38]:
del(DtNMV1)

In [39]:
np.save('NBx1', MinMaxScaler().fit_transform(DtNMIV1))
np.save('NBy1', (LNMIV1=='Benign').astype(int))

In [40]:
DtNMIV1.shape

(328181, 78)

In [41]:
MCDt1=DtNMIV1[LNMIV1 !='Benign',:]

In [42]:
MCDt1.shape

(92403, 78)

In [43]:
import numpy as np
import os
import tensorflow as tf
import keras
from keras import models
from keras import layers

In [44]:
x=np.load('NBx1.npy')
y=np.load('NBy1.npy')


x=x.astype(np.float32)

In [45]:
x=np.delete(x,0,1)

In [46]:
(unique, counts) = np.unique(y, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[     0  92403]
 [     1 235778]]


In [47]:
x.shape

(328181, 77)

In [48]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=123)

In [49]:
def create_model():
  model = models.Sequential()
  model.add(layers.Dense(256,input_dim=x.shape[1],activation='relu',use_bias=True))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(128,activation='relu',use_bias=True))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(64,activation='relu',use_bias=True))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(32,activation='relu',use_bias=True))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(1,activation='sigmoid',use_bias=True)) 
  return model
 
model = create_model()
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
model.summary()
history=model.fit(xtrain , ytrain ,batch_size=1024 , epochs=20 ,verbose=1 ,shuffle=True)
result = model.evaluate(xtest,ytest)
dict(zip(model.metrics_names, result))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 256)               19968     
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 32)               

{'accuracy': 0.7794536352157593, 'loss': 0.5053901076316833}

In [50]:
y_pred=model.predict(xtest)

In [51]:
from sklearn import metrics
print("The accuracy score is",metrics.accuracy_score(ytest, y_pred.round()))
print("The precision score is",metrics.precision_score(ytest, y_pred.round(),average='weighted'))
print("The recall score is",metrics.recall_score(ytest, y_pred.round(),average='weighted'))
print("The f1-score is",metrics.f1_score(ytest, y_pred.round(),average='weighted'))
print('\n')
print(metrics.confusion_matrix(ytest, y_pred.round()))
print('\n')
print("=== Classification Report ===")
print(metrics.classification_report(ytest, y_pred.round()))
print('\n')

The accuracy score is 0.779453661806603
The precision score is 0.8106854998356411
The recall score is 0.779453661806603
The f1-score is 0.7291406682306085


[[ 4489 14076]
 [  400 46672]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.92      0.24      0.38     18565
           1       0.77      0.99      0.87     47072

    accuracy                           0.78     65637
   macro avg       0.84      0.62      0.62     65637
weighted avg       0.81      0.78      0.73     65637





In [53]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc_history=rfc.fit(xtrain,ytrain)
rfc_predict = rfc.predict(xtest)

In [54]:
from sklearn import metrics
print("The accuracy score is",metrics.accuracy_score(ytest, rfc_predict))
print("The precision score is",metrics.precision_score(ytest, rfc_predict,average='weighted'))
print("The recall score is",metrics.recall_score(ytest, rfc_predict,average='weighted'))
print("The f1-score is",metrics.f1_score(ytest, rfc_predict,average='weighted'))
print('\n')
print(metrics.confusion_matrix(ytest, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(metrics.classification_report(ytest, rfc_predict))
print('\n')

The accuracy score is 0.7652543534896477
The precision score is 0.7509663686790712
The recall score is 0.7652543534896477
The f1-score is 0.7334572070148475


[[ 5881 12684]
 [ 2724 44348]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.68      0.32      0.43     18565
           1       0.78      0.94      0.85     47072

    accuracy                           0.77     65637
   macro avg       0.73      0.63      0.64     65637
weighted avg       0.75      0.77      0.73     65637



